In [8]:
import pandas as pd
from datetime import datetime
import random
import pickle
from tqdm import tqdm_notebook  

In [9]:
event_types={
    "px_medications_injections_infusions_and_other_forms_":0,
    "dx_brnch_lng_ca":1,
    "px_pathology":2,
    "px_therapeutic_radiology":3,
    "px_laboratory_chemistry_and_hematology":4,
    "px_ct_scan_abdomen":5,
    "dx_ot_perint_dx":6,
    "px_other_diagnostic_procedures_on_lung_and_bronchus":7,
    "dx_2ndary_malig":8,
    "px_ct_scan_chest":9,
    "dx_anemia":10,
    "px_radioisotope_scan_and_function_studies":11,
    "px_other_ct_scan":12,
    "px_magnetic_resonance_imaging":13,
    "px_diagnostic_bronchoscopy_and_biopsy_of_bronchus":14,
    "px_lobectomy_or_pneumonectomy":15,
    "px_dme_and_supplies":16,
    "rx_antineoplastic_agents":17,
    "dx_fx_arm":18,
    "px_incision_of_pleura_thoracentesis_chest_drainage":19    
}

In [10]:
from datetime import date

def timediff(date1,date2):
    end_date = (date2)
    start_date = (date1)
    num_days = (end_date-start_date).days 
    if (num_days==0):
        return None
    return num_days

In [11]:
def get_all_patients(df_name):
    unique_patients=set()
    df1=pd.read_csv(df_name)
    for i in range(len(df1)):
        unique_patients.add(df1.loc[i,"person_id"])
    return unique_patients

In [12]:
# dates = df[df['person_id']=='a']['event_date'].tolist()
# dates.sort(key=lambda date: datetime.strptime(date, "%d/%m/%Y"))

In [6]:
def get_data(df,patient_id):
    patient_df = df[df['person_id']==patient_id]
    patient_df['event_date'] = pd.to_datetime(patient_df['event_date'])
    patient_df = patient_df.sort_values(by='event_date')
    start_date = patient_df['event_date'].min()
    patient_df['last_event_dt'] = patient_df['event_date'].shift(1).fillna(method='bfill')
    patient_df[['event_date','last_event_dt']]
    patient_df['time_since_last_event'] = patient_df.apply(lambda x: timediff(x['last_event_dt'],x['event_date']),axis=1)
    patient_df[['event_date','last_event_dt','time_since_last_event']]
    patient_df['time_since_start'] = patient_df.apply(lambda x: timediff(start_date,x['event_date']),axis=1)
    patient_df['idx_event'] = [i for i in range(1,patient_df.shape[0]+1)]
    patient_df['type_event'] = patient_df['event_label'].apply(lambda x: event_types[x])
    patient_df = patient_df[['person_id','idx_event','type_event','time_since_start','time_since_last_event']].rename(columns={'person_id':'patient_id'})
    patient_df['time_since_start'].fillna(0,inplace=True)
    patient_df['time_since_last_event'].fillna(0,inplace=True)
    patient_df = patient_df.dropna(subset=['time_since_last_event'])
    l = list(patient_df.T.to_dict().values())
    return l

C:\Users\am21907\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
pd.set_option('display.max_rows',None)
# patient_df[patient_df['same_day_event_rank'] >=1]
patient_df

,patient_id,idx_event,type_event,time_since_start,time_since_last_event,same_day_event_rank,time_since_start_adj,time_since_last_event_adj
2942,1073759707497,1,1,0.0,0.0,0,0.00,0.00
3357,1073759707497,2,1,1.0,1.0,0,1.00,1.00
19461,1073759707497,3,1,2.0,1.0,0,2.00,1.00
9622,1073759707497,4,1,3.0,1.0,0,3.00,1.00
9073,1073759707497,5,4,10.0,7.0,0,10.00,7.00
727,1073759707497,6,1,10.0,0.0,1,10.01,0.01
19875,1073759707497,7,0,21.0,11.0,0,21.00,11.00
19084,1073759707497,8,0,21.0,0.0,1,21.01,0.01
1696,1073759707497,9,4,21.0,0.0,2,21.02,0.02
22205,1073759707497,10,1,21.0,0.0,3,21.03,0.03


In [17]:
# patient_id = 1073759707497
# patient_df = df[df['person_id']==patient_id]
# patient_df['event_date'] = pd.to_datetime(patient_df['event_date'])
# patient_df = patient_df.sort_values(by='event_date')
# start_date = patient_df['event_date'].min()
# patient_df['last_event_dt'] = patient_df['event_date'].shift(1).fillna(method='bfill')
# patient_df[['event_date','last_event_dt']]
# patient_df['time_since_last_event'] = patient_df.apply(lambda x: timediff(x['last_event_dt'],x['event_date']),axis=1)
# patient_df[['event_date','last_event_dt','time_since_last_event']]
# patient_df['time_since_start'] = patient_df.apply(lambda x: timediff(start_date,x['event_date']),axis=1)
# patient_df['idx_event'] = [i for i in range(1,patient_df.shape[0]+1)]
# patient_df['type_event'] = patient_df['event_label'].apply(lambda x: event_types[x])
# patient_df = patient_df[['person_id','idx_event','type_event','time_since_start','time_since_last_event']].rename(columns={'person_id':'patient_id'})
# l = list(patient_df.T.to_dict().values())

In [18]:
# unique_patients=get_all_patients()
# seqs=[]
# for x in unique_patients:
#     seqs.append(get_data(x))
# print(seqs)

In [19]:
def find_key(dict,val):
    for key,value in dict.items():
        if(value==val):
            return key

In [20]:
def remove_data(df,patient_list,impute_rate,impute_value):
    s="event_label=={}".format(impute_value)
    df=df.drop(df[df["event_label"]==impute_value].sample(frac=impute_rate,random_state=1).index).reset_index(drop=True)
    obs_list=[]
    for x in patient_list:
        obs_list.append(get_data(df,x))
    return obs_list

In [45]:
#driver
df_name = "data/test/sample_data.csv"
train_patients=[]
test_patients=[]
dev_patients=[]
patients=get_all_patients(df_name)
patients=list(patients)
train_patients_size=0.7*len(patients)
test_patients_size=0.2*len(patients)
dev_patients_size=0.1*len(patients)
count=0
while(count<train_patients_size):
    train_patients.append(patients[count])
    count+=1
while(count<train_patients_size+test_patients_size):
    test_patients.append(patients[count])
    count+=1
while(count<len(patients)):
      dev_patients.append(patients[count])
      count+=1
#train data is ready
df=pd.read_csv(df_name)
# print(df)

train_seqs=[]
test_seqs=[]
dev_seqs=[]
      
for x in tqdm_notebook(train_patients):
    temp = get_data(df,x)
    if temp:
        train_seqs.append(temp)
for x in tqdm_notebook(test_patients):
    temp = get_data(df,x)
    if temp:
        test_seqs.append(temp)
for x in tqdm_notebook(dev_patients):
    temp = get_data(df,x)
    if temp:
        dev_seqs.append(temp)
        
key=find_key(event_types,9)

train_obs=remove_data(df,train_patients,1,key)
test_obs=remove_data(df,test_patients,1,key)
dev_obs=remove_data(df,dev_patients,1,key)

# key=find_key(event_types,)

# train_obs=remove_data(df,train_patients,1,key)
# test_obs=remove_data(df,test_patients,1,key)
# dev_obs=remove_data(df,dev_patients,1,key)


C:\Users\am21907\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


C:\Users\am21907\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\am21907\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


KeyboardInterrupt: 

In [22]:
# pd.DataFrame(seqs[0])
for sequence in dev_obs:
    if sequence[0]['patient_id']==592713554906:
        display(pd.DataFrame(sequence))
        

,patient_id,idx_event,type_event,time_since_start,time_since_last_event
0,5.927136e+11,2.0,7.0,41.0,41.0
1,5.927136e+11,3.0,7.0,48.0,7.0
2,5.927136e+11,5.0,7.0,132.0,84.0
3,5.927136e+11,6.0,2.0,153.0,21.0
4,5.927136e+11,10.0,2.0,155.0,2.0
...,...,...,...,...,...
97,5.927136e+11,236.0,1.0,1977.0,2.0
98,5.927136e+11,238.0,8.0,1978.0,1.0
99,5.927136e+11,241.0,1.0,1979.0,1.0
100,5.927136e+11,242.0,8.0,1994.0,15.0


In [23]:
for sequence in dev_seqs:
    if sequence[0]['patient_id']==592713554906:
        display(pd.DataFrame(sequence))

,patient_id,idx_event,type_event,time_since_start,time_since_last_event
0,5.927136e+11,2.0,7.0,41.0,41.0
1,5.927136e+11,3.0,7.0,48.0,7.0
2,5.927136e+11,5.0,7.0,132.0,84.0
3,5.927136e+11,6.0,4.0,153.0,21.0
4,5.927136e+11,11.0,14.0,155.0,2.0
...,...,...,...,...,...
98,5.927136e+11,241.0,1.0,1977.0,2.0
99,5.927136e+11,243.0,8.0,1978.0,1.0
100,5.927136e+11,246.0,1.0,1979.0,1.0
101,5.927136e+11,247.0,1.0,1994.0,15.0


In [24]:
def store_as_pickle(seqs,seqs_obs,total_num,file_name):
    if len(seqs) == len(seqs_obs):
        final_data = {
            'seqs': seqs,
            'seqs_obs': seqs_obs,
            'total_num':total_num
        }
        f = open("data/test/{}".format(file_name),'wb') 
        # source, destination 
        pickle.dump(final_data, f)                      
        f.close() 
    else:
        print("shape does not match")

In [25]:
store_as_pickle(train_seqs,train_obs,20,"train.pkl")
store_as_pickle(test_seqs,test_obs,20,"test.pkl")
store_as_pickle(dev_seqs,dev_obs,20,"dev.pkl")